# Importar librerias

Una vez realizado el analisis de las variables, se identifico que el objeto del contrato es la variable que contiene mayor información para poder determinar el sector del contrato. Por esta razón, se realiza el procesamiento de la variable a continuación:

In [9]:
import os
import sys

import pandas as pd
import numpy as np
import unidecode
import re
import spacy


# librerias propias
dir_proyecto, _ = os.path.split(os.getcwd())
sys.path.append(dir_proyecto)
nlp = spacy.load("es_core_news_sm")


In [17]:
from funciones import *

In [18]:
# Cargamos los datos

cont_df = leer_parquet(os.path.join(dir_proyecto,'INPUTS'))
df = cont_df[~cont_df['objeto_del_contrato'].isna()].reset_index(drop=True)

In [20]:
df.head(5)

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,recursos_propios,ultima_actualizacion,codigo_entidad,fecha_inicio_liquidacion,fecha_fin_liquidacion,codigo_proveedor,objeto_del_contrato,fecha_de_inicio_del_contrato,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion
0,MUNICIPIO SAN JUAN NEPOMUCENO,800037175,Bolívar,San Juan Nepomuceno,"Colombia, Bolívar , San Juan Nepomuceno",Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,CO1.BDOS.4792282,...,0,2023-08-10T00:00:00.000,704441443,2023-12-11 17:00:00,2024-04-10 17:00:00,707948352,AUNAR ESFUERZOS Y RECURSOS PARA LA RECUPERACIO...,None,NaN,NaN
1,UNIDAD ADMINISTRATIVA ESPECIAL DE LA JUSTICIA ...,900384997,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,defensa,Ejecutivo,Centralizada,CO1.BDOS.4460789,...,0,2023-11-01T00:00:00.000,700960073,2023-12-16 17:00:00,2024-04-16 17:00:00,710522970,Adquisición de mobiliario para la Unidad Admin...,2023-08-22T00:00:00.000,NaN,NaN
2,SENA REGIONAL HUILA Grupo Administrativo CADP,899999034,Huila,Garzón,"Colombia, Huila , Garzón",Nacional,Trabajo,Ejecutivo,Descentralizada,CO1.BDOS.4669840,...,0,2023-08-22T00:00:00.000,704161496,2023-12-16 17:00:00,2024-04-16 23:00:00,702496043,SUMINISTRAR MATERIALES DE FORMACIÓN PARA EL ÁR...,2023-08-14T00:00:00.000,NaN,NaN
3,UNIDAD DE SALUD DE IBAGUE USI ESE,809003590,Tolima,No Definido,"Colombia, Tolima",Territorial,Salud y Protección Social,Ejecutivo,Descentralizada,CO1.BDOS.4822351,...,0,2023-10-24T00:00:00.000,700949019,None,None,720020171,CONTRATAR LA PRESTACION DE SERVICIOS DE ACTIVI...,2023-08-14T00:00:00.000,NaN,NaN
4,INSTITUTO NACIONAL DE VIAS,800215807,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,Transporte,Ejecutivo,Centralizada,CO1.BDOS.4603978,...,0,2023-10-06T00:00:00.000,700676059,2024-01-01 17:00:00,2024-06-30 17:00:00,721263846,ATENDER SITIOS CRITICOS DE LA RED TERCIARIA A ...,2023-09-04T00:00:00.000,NaN,NaN


In [21]:
def limpiar_texto(string):
    
    #Estandarizar acentuaciones
    out = unidecode.unidecode(string)

    #Poner en minúsculas
    out = out.lower()
    
    #Eliminar caracteres que no sean del alfabeto básico latino [A-Za-z0-9_] espacios y saltos de línea
    out = re.sub(r"[^\w\s]|\n", ' ', out)
    
    #Eliminar uno o varios dígitos seguidos
    out = re.sub("\d+", "", out)
    
    #Eliminar todos los espacios seguidos luego de reemplazar el texto
    out = re.sub('\s+', ' ', out)

    #NLP object
    out = nlp(out)
    #Eliminar palabras de parada 'stopwords'
    out = [token.text for token in out if not token.is_stop]
    out = " ".join(out)

    #Lematizar
    lemmas =[token.lemma_ for token in nlp(out)]
    # Convertir la lista de lemmas nuevamente a texto
    out = " ".join(lemmas)

    #Remover artículos (el/la/lo/etc.) o palabras de dos letras")
    out = [token.text for token in nlp(out) if len(token) > 2]

    return out


In [23]:
df ["objeto_del_contrato"] = df ["objeto_del_contrato"].apply(limpiar_texto)

In [25]:
df["objeto_del_contrato"].head(10)

0    [aunar, esfuerzo, recurso, recuperacion, cober...
1    [adquisicion, mobiliario, unidad, administrati...
2    [suministrar, material, formacion, area, agrop...
3    [contratar, prestacion, servicio, actividadir,...
4    [atender, sitio, critico, red, terciario, carg...
5    [contratisto, obligar, suministrar, asistencia...
6    [construccion, pista, pumptrack, nino, constru...
7    [analisis, viabilidad, nutricional, inclusion,...
8    [brindar, apoyo, corporacion, cultura, turismo...
9    [mejoramiento, locativo, modalidad, monto, ago...
Name: objeto_del_contrato, dtype: object

In [30]:
data = df ["objeto_del_contrato"]

data.to_csv("../Data/muestra_sin_contexto.csv", index=False)